# Importando as bibliotecas


In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import datasets

In [ ]:
#instala e import o apriori
!pip install apyori
from apyori import apriori

  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=08d718e59e780f27f8babced379e798cc369c908a64c3a45714fce3b6186022d
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


# Criando o DataFrame de IRIS

In [ ]:
#Obtendo o Dataset
data = datasets.load_iris()
 
# create sample dataset
columns = ['sepal length','sepal width','petal length','petal width']
dataFrameIris = pd.DataFrame(data['data'], columns=columns) 

# alterando os valores numericos para categoricos
novaListaTarget = []
for i in range(0,len(list(data['target']))):
  if data['target'][i] == 0:
     novaListaTarget.append("Iris Setosa")
  elif data['target'][i] == 1:
     novaListaTarget.append("Iris Versicolour")
  else:
     novaListaTarget.append("Iris Virginica")    
  
dataFrameIris['class'] = novaListaTarget;
dataFrameIris

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris Setosa
1,4.9,3.0,1.4,0.2,Iris Setosa
2,4.7,3.2,1.3,0.2,Iris Setosa
3,4.6,3.1,1.5,0.2,Iris Setosa
4,5.0,3.6,1.4,0.2,Iris Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris Virginica
146,6.3,2.5,5.0,1.9,Iris Virginica
147,6.5,3.0,5.2,2.0,Iris Virginica
148,6.2,3.4,5.4,2.3,Iris Virginica


# Definindo funções 

In [ ]:
from functools import partial

def dataFrameToList(dataFrame):
  linhas, cols = dataFrame.shape
  transactions = []
  for i in range(0, linhas):
    transactions.append([str(dataFrame.values[i, j]) for j in range(0, cols)])
  return transactions  


def exibirResultado(lista):
  for i in range(len(lista)):
    LHS=list(lista[i][2][0][0]) #antecedente
    RHS=list(lista[i][2][0][1]) #consequente
    support=lista[i][1]
    confidence=lista[i][2][0][2]
    lift=lista[i][2][0][3]

    print(LHS, "->", RHS)
    print("Suporte", support)
    print("confianca", confidence)
    print("Lift:", lift)
    print(10*"----------")  


def criandoFaixasDeAcordoComAMedia(dataFrame, nomeColuna):
  valorMax = max(dataFrame[nomeColuna])
  valorMin = min(dataFrame[nomeColuna])

  mediaValores = sum([valorMax, valorMin])/2
  novaLista = []
  for i in range(0, len(dataFrame[nomeColuna])):
    if dataFrame[nomeColuna][i] >= mediaValores:
      novaLista.append('faixa_maior_media'+nomeColuna)
    else:
      novaLista.append('faixa_menor_media'+nomeColuna)
  return novaLista  



def criandoFaixasDeAcordoComPorcentagem(dataFrame, nomeColuna):
  setentaCincoPorcento = dataFrame[nomeColuna].describe()[6:7][0]
  cinquentaPorcento = dataFrame[nomeColuna].describe()[5:6][0]
  vinteCincoPorcento = dataFrame[nomeColuna].describe()[4:5][0]

  novaLista = []
  for i in range(0, len(dataFrame[nomeColuna])):
    if dataFrame[nomeColuna][i] >= setentaCincoPorcento:
      novaLista.append(nomeColuna+':acima75')
    elif dataFrame[nomeColuna][i] >= cinquentaPorcento:
      novaLista.append(nomeColuna+':entre50e75') 
    elif dataFrame[nomeColuna][i] >= vinteCincoPorcento:
      novaLista.append(nomeColuna+':entre25e50')  
    else:
      novaLista.append(nomeColuna+':menor25')
  return novaLista   

# Executando o Apriori sem transformar os dados e modificando o minsup e minconf


In [ ]:
dataFrameIrisList = dataFrameToList(dataFrameIris) 
rules = list(apriori(dataFrameIrisList, min_support=0.3, min_confidence=0.3))
pd.DataFrame(rules)

,items,support,ordered_statistics
0,(Iris Setosa),0.333333,"[((), (Iris Setosa), 0.3333333333333333, 1.0)]"
1,(Iris Versicolour),0.333333,"[((), (Iris Versicolour), 0.3333333333333333, ..."
2,(Iris Virginica),0.333333,"[((), (Iris Virginica), 0.3333333333333333, 1.0)]"


In [ ]:
print(10*"----------")
exibirResultado(rules)

----------------------------------------------------------------------------------------------------
[] -> ['Iris Setosa']
Suporte 0.3333333333333333
confianca 0.3333333333333333
Lift: 1.0
----------------------------------------------------------------------------------------------------
[] -> ['Iris Versicolour']
Suporte 0.3333333333333333
confianca 0.3333333333333333
Lift: 1.0
----------------------------------------------------------------------------------------------------
[] -> ['Iris Virginica']
Suporte 0.3333333333333333
confianca 0.3333333333333333
Lift: 1.0
----------------------------------------------------------------------------------------------------


Não foi localizado associação com suporte maior de 40%, quando alterado para  suporte para 30% é retornado os itens sem associação 

# Executando apriori com transformação dos dados utilizando media entre valor maximo da coluna e o valor minimo


In [ ]:
#Tratando os dados e efetuando as transformacoes 
#Nesse caso optei por transforma de acordo com a media entre o valor minimo e o valor maximo
#quem fica acima da media entra em uma faixa, quem fica abaixo entra em outra faixa

dataFrameIrisTratamento = pd.DataFrame([], columns=columns)   

dataFrameIrisTratamento['sepal length'] = criandoFaixasDeAcordoComAMedia(dataFrameIris,'sepal length')
dataFrameIrisTratamento['sepal width'] = criandoFaixasDeAcordoComAMedia(dataFrameIris,'sepal width')
dataFrameIrisTratamento['petal length'] = criandoFaixasDeAcordoComAMedia(dataFrameIris,'petal length')
dataFrameIrisTratamento['petal width'] = criandoFaixasDeAcordoComAMedia(dataFrameIris,'petal width')
dataFrameIrisTratamento['class'] = dataFrameIris['class']

dataFrameIrisTratamento


,sepal length,sepal width,petal length,petal width,class
0,faixa_menor_mediasepal length,faixa_maior_mediasepal width,faixa_menor_mediapetal length,faixa_menor_mediapetal width,Iris Setosa
1,faixa_menor_mediasepal length,faixa_menor_mediasepal width,faixa_menor_mediapetal length,faixa_menor_mediapetal width,Iris Setosa
2,faixa_menor_mediasepal length,faixa_maior_mediasepal width,faixa_menor_mediapetal length,faixa_menor_mediapetal width,Iris Setosa
3,faixa_menor_mediasepal length,faixa_menor_mediasepal width,faixa_menor_mediapetal length,faixa_menor_mediapetal width,Iris Setosa
4,faixa_menor_mediasepal length,faixa_maior_mediasepal width,faixa_menor_mediapetal length,faixa_menor_mediapetal width,Iris Setosa
...,...,...,...,...,...
145,faixa_maior_mediasepal length,faixa_menor_mediasepal width,faixa_maior_mediapetal length,faixa_maior_mediapetal width,Iris Virginica
146,faixa_maior_mediasepal length,faixa_menor_mediasepal width,faixa_maior_mediapetal length,faixa_maior_mediapetal width,Iris Virginica
147,faixa_maior_mediasepal length,faixa_menor_mediasepal width,faixa_maior_mediapetal length,faixa_maior_mediapetal width,Iris Virginica
148,faixa_maior_mediasepal length,faixa_maior_mediasepal width,faixa_maior_mediapetal length,faixa_maior_mediapetal width,Iris Virginica


In [ ]:
#Executando o apriori
dataFrameIrisTratamentoList = dataFrameToList(dataFrameIrisTratamento) 

rulesTratamento = list(apriori(dataFrameIrisTratamentoList, min_support=0.5, min_confidence=0.9))
pd.DataFrame(rulesTratamento)

,items,support,ordered_statistics
0,"(faixa_maior_mediapetal width, faixa_maior_med...",0.553333,"[((faixa_maior_mediapetal length), (faixa_maio..."


In [ ]:
exibirResultado(rulesTratamento)

['faixa_maior_mediapetal length'] -> ['faixa_maior_mediapetal width']
Suporte 0.5533333333333333
confianca 0.9325842696629213
Lift: 1.6457369464639788
----------------------------------------------------------------------------------------------------


Nesse caso o retorno foi ao menos estranho hahaha a ideia era separar em duas faixas, acima da media e abaixo da media, a media foi calculado de acordo com o valor min e o valor maximo de cada coluna. Utilizando o suporte maior de 50% e a confiança de 90% é retornado os valores das faixas e não os itens.

# Executando apriori com transformação dos dados utilizando faixas de porcentagem do describe()



In [ ]:
#Tratando os dados e efetuando as transformacoes 
#Nesse caso optei por transforma de acordo as porcentagem de 25%, 50% e 75%.
#Ao todo foram criado 4 faixas
# 1 - Acima de 75%
# 2 - Entre 50% e 75%
# 3 - Entre 25% e 50%
# 4 - Abaixo de 25%

dataFrameTratamentoPorcentagem = pd.DataFrame([], columns=columns)   

dataFrameTratamentoPorcentagem['sepal length'] = criandoFaixasDeAcordoComPorcentagem(dataFrameIris,'sepal length')
dataFrameTratamentoPorcentagem['sepal width'] = criandoFaixasDeAcordoComPorcentagem(dataFrameIris,'sepal width')
dataFrameTratamentoPorcentagem['petal length'] = criandoFaixasDeAcordoComPorcentagem(dataFrameIris,'petal length')
dataFrameTratamentoPorcentagem['petal width'] = criandoFaixasDeAcordoComPorcentagem(dataFrameIris,'petal width')
dataFrameTratamentoPorcentagem['class'] = dataFrameIris['class']

dataFrameTratamentoPorcentagem

,sepal length,sepal width,petal length,petal width,class
0,sepal length:entre25e50,sepal width:acima75,petal length:menor25,petal width:menor25,Iris Setosa
1,sepal length:menor25,sepal width:entre50e75,petal length:menor25,petal width:menor25,Iris Setosa
2,sepal length:menor25,sepal width:entre50e75,petal length:menor25,petal width:menor25,Iris Setosa
3,sepal length:menor25,sepal width:entre50e75,petal length:menor25,petal width:menor25,Iris Setosa
4,sepal length:menor25,sepal width:acima75,petal length:menor25,petal width:menor25,Iris Setosa
...,...,...,...,...,...
145,sepal length:acima75,sepal width:entre50e75,petal length:acima75,petal width:acima75,Iris Virginica
146,sepal length:entre50e75,sepal width:menor25,petal length:entre50e75,petal width:acima75,Iris Virginica
147,sepal length:acima75,sepal width:entre50e75,petal length:acima75,petal width:acima75,Iris Virginica
148,sepal length:entre50e75,sepal width:acima75,petal length:acima75,petal width:acima75,Iris Virginica


In [ ]:
#Executando o apriori
dataFrameTratamentoPorcentagemList = dataFrameToList(dataFrameTratamentoPorcentagem) 

rulesTratamentoPorcentagemList = list(apriori(dataFrameTratamentoPorcentagemList, min_support=0.3, min_confidence=0.4))
pd.DataFrame(rulesTratamentoPorcentagemList)

,items,support,ordered_statistics
0,"(Iris Virginica, petal width:acima75)",0.3,"[((Iris Virginica), (petal width:acima75), 0.9..."


In [ ]:
exibirResultado(rulesTratamentoPorcentagemList)

['Iris Virginica'] -> ['petal width:acima75']
Suporte 0.3
confianca 0.9
Lift: 2.9347826086956523
----------------------------------------------------------------------------------------------------


Aqui já tivemos um retorno melhor, a ideia era separar em 4 faixas:

1.   Acima de 75%
2.   Entre 50% e 75%
3.   Entre 25% e 50% 
4.   Abaixo de 25%

Utilizando um suporte de 30% e uma confiança de 40%, foi retornado que:

```
# Iris Virginica -> petal width:acima75
```